# LLM Society: Complete User Guide (English)

This notebook teaches how to use the `llm-society` package end-to-end:

- Installation and API key setup
- Quick start (simple and complex contagion modes)
- Object-oriented API (LLM mode)
- Visualizations via a single entrypoint `net.plot(type=...)`
- Persona segments
- Scheduled interventions with `intervention_content`
- Using a custom NetworkX graph (real data)
- Conversation retrieval (by round and node pair)
- Summary panel and exporting results
- CLI usage
- Tips for cost control and reproducibility



## 1) Installation and setup

```bash
pip install llm-society
```

LLM mode requires an OpenAI API key:
```bash
export OPENAI_API_KEY=<YOUR_OPENAI_API_KEY>
# Or save to file (gitignored):
echo "<YOUR_OPENAI_API_KEY>" > api-key.txt
```
The library reads from `OPENAI_API_KEY` or `OPENAI_API_KEY_FILE`.



In [ ]:
# Imports
from llm_society import network
import matplotlib.pyplot as plt



## 2) Quick start: simple and complex contagion
Use non-LLM modes for fast demos and theory comparisons.



In [ ]:
# Simple contagion
net_simple = network(
    information="Increasing the carbon tax will reduce emissions.",
    n=16, degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.9,
    mode="simple", rng=0,
)
net_simple.simulate()
net_simple.plot(type="coverage")

# Complex contagion (threshold k)
net_complex = network(
    information="Public transit expansion will reduce traffic congestion.",
    n=16, degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.9,
    mode="complex", rng=0,
)
net_complex.simulate()
net_complex.plot(type="coverage")



## 3) Stepwise simulation (pause and resume)
`step()` advances one round at a time and accumulates `history`.



In [ ]:
net_step = network(
    information="Urban greening improves public health.",
    n=14, degree=3, rounds=10,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.9,
    mode="simple", rng=1,
)
count = 0
while net_step.step():
    count += 1
    if count >= 3:
        break
net_step.plot(type="coverage")



## 4) LLM mode (conversations + belief updates)
This mode uses short, natural-sounding conversations to update beliefs.



In [ ]:
net_llm = network(
    information="Expanding vocational training will increase employment.",
    n=20, degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=0, model="gpt-4.1",
)
net_llm.simulate()
net_llm.plot(type="coverage")
net_llm.plot(type="final_beliefs")



## 5) Visualizations with a single entrypoint
Available plot `type`s:
- `animation`, `coverage`, `final_beliefs`
- `group_beliefs` (kwargs: `attr="political"`)
- `centrality` (kwargs: `metric="degree"|"betweenness"|"eigenvector"`)
- `intervention_effect` (kwargs: `intervention_round=int`, optional; auto-detected if omitted)



In [ ]:
# Group means and centrality plots
net_llm.plot(type="group_beliefs", attr="political")
net_llm.plot(type="centrality", metric="betweenness")



## 6) Persona segments
Segments allow you to define sub-populations with different trait distributions.



In [ ]:
segments = [
    {"proportion": 0.5, "traits": {"political": {"choices": {"Democrat": 0.8, "Republican": 0.2}}}},
    {"proportion": 0.5, "traits": {"political": {"choices": {"Democrat": 0.2, "Republican": 0.8}}}},
]

net_seg = network(
    information="Increasing gas taxes will significantly reduce inflation.",
    n=24, degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=1, model="gpt-4.1",
    segments=segments,
)
net_seg.simulate()
net_seg.plot(type="group_beliefs", attr="political")



## 7) Scheduled interventions (content injection)
From a given round onward, inject `intervention_content` into the system message of targeted nodes.



In [ ]:
net_int = network(
    information="Increasing gas taxes will significantly reduce inflation.",
    n=24, degree=4, rounds=10,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=0, model="gpt-4.1",
    intervention_round=6,
    intervention_nodes=[2,5,7,11,13,17],
    intervention_content="Write a letter to each other.",
)
net_int.simulate()
net_int.plot(type="intervention_effect", intervention_round=6)



## 8) Custom NetworkX graph (real data)
You can pass any NetworkX graph; nodes are relabeled to 0..N-1 if needed.



In [ ]:
import networkx as nx

G = nx.karate_club_graph()
net_custom = network(
    information="Raising the minimum wage will rapidly reduce unemployment.",
    degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=1, model="gpt-4.1",
    graph=G,
)
net_custom.simulate()
net_custom.plot(type="coverage")



## 9) Conversation retrieval
All per-edge conversations are stored per round in LLM mode.



In [ ]:
# Use the LLM run above
convos_r5 = net_llm.conversations(round=5)
print(f"Round 5 conversations: {len(convos_r5)} edges")

rec = net_llm.get_conversation(round=5, u=2, v=7)
if rec:
    print("Did talk:", rec["did_talk"]) 
    print("Sample turns:")
    for line in rec["turns"][:6]:
        print(line)
else:
    print("No record for (2,7) at round 5")



## 10) Summary panel and exporting results
Use `summary()` for quick metrics and `export()` for CSV/JSONL outputs.



In [ ]:
print(net_llm.summary())

# Export to files (uncomment to write files)
# net_llm.export(
#     history_csv="history_user_guide.csv",
#     beliefs_csv="beliefs_user_guide.csv",
#     conversations_jsonl="conversations_user_guide.jsonl",
# )



## 11) CLI usage

Write an example config:
```bash
llm-society --write-example-config my-config.yaml
```
Run a simulation from config:
```bash
llm-society --config my-config.yaml
```
Override parameters via flags:
```bash
llm-society --config my-config.yaml --depth 0.8 --rounds 20
```
Run fully via flags (with intervention content):
```bash
llm-society \
  --information "Increasing gas taxes will significantly reduce inflation." \
  --n 24 --degree 4 --rounds 10 \
  --depth 0.5 --depth-max 4 --edge-frac 0.5 \
  --seeds 0,1 --seed-belief 0.95 --talk-prob 0.5 \
  --mode llm --rng 0 --model gpt-4.1 \
  --intervention-round 6 --intervention-nodes 2,5,7,11,13,17 \
  --intervention-content "Write a letter to each other."
```



## 12) Tips for cost control and reproducibility
- Use `mode="simple"` or `mode="complex"` for dry-runs.
- Reduce `n`, `rounds`, `edge_frac`, `depth`/`max_convo_turns`, and `talk_prob` to cut LLM calls.
- Set `rng` to a fixed seed for reproducibility.
- Prefer `group_beliefs`, `coverage` for fast, high-signal visual summaries.

